In [7]:
import numpy as np
import pygame
from Cuerpo import cuerpo
import pickle
from tensorflow import keras

In [8]:
with open("listos\genios.pkl","rb") as genius:
    genios=pickle.load(genius)

In [9]:
len(genios)

51

In [10]:
class cerebro:
    
    def __init__(self,entrada=8,salida=4):
        
        #cantidad de datos de entrada de la RNA
        self.entrada=entrada
        #cantidad de datos de salida de la RNA
        self.salida=salida   
        
        #se guardan las dimenciones de las matrices de la RNA
        self.estructura=[] 
        #se guarda el modelo de la RNA por individuo
        self.brain=self.model()
        
        
    def model(self):
        
        model_Q=keras.models.Sequential()
        model_Q.add(keras.layers.Dense(units=8,activation="tanh",input_dim=self.entrada,use_bias=False))
        model_Q.add(keras.layers.Dense(units=18,activation="tanh",use_bias=False))
        model_Q.add(keras.layers.Dense(units=18,activation="tanh",use_bias=False))
        #model_Q.add(keras.layers.Dropout(0.5))
        #model_Q.add(keras.layers.Dense(units=8,activation="tanh",use_bias=False))
        model_Q.add(keras.layers.Dense(units=self.salida,activation="softmax",use_bias=False))
        model_Q.compile(loss="mse",optimizer=keras.optimizers.Adam(lr=0.001))
        
        val=[]
        for i in model_Q.weights:
            val.append(i.numpy().shape)
        self.estructura=val
        
        return model_Q
        
#funcion para escalar los datos de entrada
scaler=lambda x: x/250

class nido:
    
    def __init__(self,gin,poblacion=10):
        
        #cantidad de Snakes a crear
        self.poblacion=poblacion
        
        self.gin=gin
        
        #Snake inteligentes
        self.snake_IA=self.dar_IA()
        
        #estructura de las redes neuronales de los cerebros
        self.estructura=self.snake_IA[0].estructura
        
        #peso de la RNA de cada Snkae
        self.pesos=genios[self.gin]
        
        #se cargan los primeros pesos aleatorios
        self.set_pesos(self.pesos)
        
        #variable donde se guardará el desempeño de cada Snake
        self.desemp=[]
        
        #crea los pesos del cerebro de cada una de las Snake 
    
    #crea una RNA para cada Snake
    def dar_IA(self):
        medusa=[]
        for i in range(self.poblacion):
            medusa.append(cerebro())
        return medusa
    
    #funcion para darle los pesos a las RNA
    def set_pesos(self,pesos):
        
        self.pesos=pesos
        
        for i in range(self.poblacion):
            self.snake_IA[i].brain.set_weights(self.pesos)

In [11]:
class Snake:
    
    def __init__(self,cant_pob,genio):
        
        #cantidad de población
        self.cant_pob=cant_pob
        self.generacion=0
        self.generacion=0
        #creamos el cuerpo
        self.gusano=[]
        self.vidas=[]
        self.scores=[]
        for i in range(0,cant_pob):
            self.gusano.append(cuerpo(i+1))
            self.vidas.append(self.gusano[i].live)
            self.scores.append(self.gusano[i].life_time)
        
        #creamos la inteligencia de snake
        self.genio=genio
        self.brain=nido(self.genio,self.cant_pob) 
        
        #mejores pesos
        self.genios=[]
        #variables de Pygame
        pygame.init()
        self.run=True    
        self.fps=pygame.time.Clock()                
        self.text_color=(200,60,80)
        self.font=pygame.font.Font(None,30)
        self.display=pygame.display.set_mode(size=(550,250))
        
        
        #corre la funcion principal
        self.main()
        
        pygame.quit()
        
   # def calc_falla(self):   
    
    def deci(self,x):
        return self.brain.snake_IA[x].brain.predict_step(scaler(self.gusano[x].vision(self.gusano[x].cuerpo,self.gusano[x].kiwi_pos)))
        

    def main(self):     
                 

        
        while self.run:
            
            self.display.fill(color=(0,0,0))
            #determina cuando se cierra la ventana
            for event in pygame.event.get():
                if event.type==pygame.QUIT:
                    self.run =False
                    
                    
              ##################################################################   
              ##lectura posicion
                
            
            
            #ejecuta las acciones si el Snake esta vivo
            if self.gusano[0].live==1:

                #predice el movimineto de cada Snake
                decision=self.deci(0)                    
                self.pred_mov=np.argmax(decision)
                
                        #self.pred_mov=np.random.choice([0,1,2,3])

                        #print("decision del Snake #",i+1," es: ",self.pred_mov)

                #mueve cada Snake
                self.gusano[0].mov(self.pred_mov)

                #evalua si la snake debe morir
                self.gusano[0].muerte()

                #lleva el tiepo de vida
                self.gusano[0].life_time+=1

                #lleva el conteo de los gusanos vivos
                if self.gusano[0].live==0:
                    self.vidas[0]=0
                    self.scores[0]=self.gusano[0].life_time+(self.gusano[0].score*100)


            #### dibujar a snake


                pygame.draw.rect(self.display,(0,200,0), pygame.Rect(self.gusano[0].cabeza[0],self.gusano[0].cabeza[1],10,10))
                for pos in self.gusano[0].cuerpo[1:]:            
                    pygame.draw.rect(self.display,(200,200,200), pygame.Rect(pos[0],pos[1],10,10))


                ####grafica kiwi    
                pygame.draw.rect(self.display,(169,6,6),pygame.Rect(self.gusano[0].kiwi_pos[0],self.gusano[0].kiwi_pos[1],10,10))
            #####################################################################
            #reinicio
            
            if not(1 in self.vidas):
      
                
                
                #####################################
                ####################################
                ###################################
                print(self.gusano[0].score)
                self.gusano[0].reinicio()
                self.vidas[0]= self.gusano[0].live
            
            ####################################################################
            
            
            #### tabla de info
            pygame.draw.rect(self.display,(255,255,255), pygame.Rect(250,0,300,250))
            
            #### grafica info
            
            
            info_pos=("x: "+str(self.gusano[0].cabeza[0])+" y: "+str(self.gusano[0].cabeza[1]))
            
            info_hambre=("frames sin comer: "+str(self.gusano[0].cant_hambre))
            
            texto=self.font.render(("score: "+str(self.gusano[0].score)),0,self.text_color)
            dis_pos=self.font.render(info_pos,0,self.text_color)        
            hambre=self.font.render(info_hambre,0,self.text_color)
            #texto_gen=self.font.render(("Generacion: "+str(self.generacion)),0,self.text_color)
            
            self.display.blit(texto,(270,20))
            self.display.blit(dis_pos,(270,50))
            self.display.blit(hambre,(270,80))
            
            pygame.display.flip()
            

            self.fps.tick(30)

In [15]:
vibora=Snake(1,35)